### Crytocurrencies analyis

#### Objective
Show statistical 'analysis' on
- Random variable 'winning', where X='BTC' is the event where BTC price increase the most in pct compared to other ccys during in interval
- Random varible 'no trade', where X='BTC' is the event where BTC was not traded during an interval

#### Approach
Just explore data by counting for now 


#### Dependencies

In [1]:
# Tech
import os
import sys
import pandas as pd
import numpy as np
import pendulum

from src.models import DeltaPoint, Trade    
from src.computed import get_pricepoints_winner_count, get_all_result

In [19]:
# Get data
all_result = get_all_result()
timestamps = [pendulum.from_timestamp(ts) for ts in all_result[0]]
data_points = all_result[1]

dict_keys(['BTCUSD', 'ETHUSD', 'LTCUSD', 'XRPUSD', 'EOSUSD', 'NONE'])

In [32]:
def compare_winning_periods(interval):
    periods = [7, 30, 183, 365]
    periods_text = ['Week', 'Month', 'Half-year', 'Year']
    winners = ['BTCUSD', 'ETHUSD', 'LTCUSD', 'XRPUSD', 'EOSUSD', 'NONE']
    winners_dict = {}
    for w in winners:
        winners_dict[w] = []
        for p in periods:
            count = 0
            for i in range(p):
                count += data_points[i][str(interval)]['winners'][w]
                
            avg = count / p
                
            winners_dict[w].append(avg)
                
    df = pd.DataFrame(winners_dict, index=periods_text).round(2)
    
    return df

In [34]:
df = compare_winning_periods(60)
df.head()

,BTCUSD,ETHUSD,LTCUSD,XRPUSD,EOSUSD,NONE
Week,0.48,0.19,0.12,0.10,0.1,0.0
Month,0.48,0.16,0.10,0.16,0.1,0.0
Half-year,0.45,0.17,0.12,0.16,0.1,0.0
Year,0.44,0.18,0.12,0.16,0.1,0.0


In [26]:
0.458859424999379 + 0.21055242725671616 + 0.14881055738843552 + 0.09088879517773422 + 0.0902438419368442 + 0.0006449532408900354

0.9999999999999991

In [ ]:
# Non tech settings
analysis_interval = 15 # seconds
analysis_period_start = pendulum.parse('2017-08-01T00:00:00Z')
analysis_period_in_days = [7, 30, 91, 182, 365]
ccys = ['BTCUSD', 'ETHUSD', 'EOSUSD', 'XRPUSD', 'LTCUSD']

###### Winning counts

In [ ]:
# From db
data = []
prev_count = None
for period in analysis_period_in_days:
    start = analysis_period_start
    end = start.add(days=period)
    result = get_pricepoints_winner_count(ccys, start, end, analysis_interval, prev_count)
    data.append((period, result))
    prev_result = (start, end, prev_count)


In [ ]:
rows = [list(t[1].values()) for t in data]
df = pd.DataFrame(rows, columns=ccys, index=analysis_period_in_days).round(2)
df['sum'] = df.sum(axis=1)
df = df[ccys].div(df['sum'], axis=0).round(2)
df.index.name = 'Period (days)'
df.head(10)

In [ ]:
rows = [list(t[1].values()) for t in data]
df = pd.DataFrame(rows, columns=ccys, index=analysis_period_in_days)

df['sum'] = df.sum(axis=1)
df = df[ccys].div(df['sum'], axis=0).round(2)
df.index.name = 'Period (days)'
df.head(10)

###### No trade counts

In [ ]:
no_trade_list = []
for ccy in ccys:
    prefix = ccy.replace('USD', '')
    column = prefix + '_trades_count'
    trades_count = df[column]
    aggregated_trade_counts = trades_count.value_counts()
    no_trades_count = aggregated_trade_counts.get(0.0)
    if not no_trades_count:
        no_trades_count = 0
        
    ratio = no_trades_count / trades_count.size
        
    no_trade_list.append((prefix, no_trades_count, trades_count.size, ratio))
        
no_trade_df = pd.DataFrame(no_trade_list, columns=['Ccy', 'No Trade Count', 'Total', 'Ratio'])
no_trade_df.head()